In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import load_model
from numpy import mean

In [0]:
#initialisation
vocabulary_size = 5000
max_words = 500
embedding_size=32
batch_size = 64
num_epochs = 5
activation='sigmoid'
loss='binary_crossentropy'
optimizer='adam'
metrics=['accuracy']


In [0]:
#load the data
(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print("Training samples :",len(X_train))
print("Test samples :",len(X_test))

In [6]:
#convert the id to word in reviews -- only for understanding
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
def getWords(indexList):
  print([id2word.get(i, ' ') for i in indexList])
#print('---review index---')
#print(X_train[6])
#print('---review words---')
#print(getWords(X_train[6]))
#print('---label---')
#print(y_train[6])

1646592/1641221 [==============================] - 0s 0us/step


In [0]:
#print("Before padding ",len(X_train[0]))

X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
#print("After padding ",len(X_train[0]))

In [67]:
#define model
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
#model.add(Dropout(0.5))
model.add(LSTM(units = 100))
model.add(Dropout(0.2))
#model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation=activation))
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#specify the loss function, optimizer and evaluation metrics 
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [69]:
#training set
X_training, y_training = X_train[batch_size:], y_train[batch_size:]
#validation set
X_validation, y_validation = X_train[:batch_size], y_train[:batch_size]
print("Training samples : ",len(X_training))
print("Validation samples : ",len(X_validation))

Training samples :  24936
Validation samples :  64


In [70]:
#Training
histories = model.fit(X_training, y_training, validation_data=(X_validation, y_validation), batch_size=batch_size, epochs=num_epochs)
model.save('sentiment_analysis.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24936 samples, validate on 64 samples
Epoch 1/5
24936/24936 [==============================] - 245s 10ms/step - loss: 0.4566 - accuracy: 0.7821 - val_loss: 0.3930 - val_accuracy: 0.8594
Epoch 2/5
24936/24936 [==============================] - 251s 10ms/step - loss: 0.3162 - accuracy: 0.8754 - val_loss: 0.3967 - val_accuracy: 0.8438
Epoch 3/5
24936/24936 [==============================] - 254s 10ms/step - loss: 0.2638 - accuracy: 0.8980 - val_loss: 0.2277 - val_accuracy: 0.9531
Epoch 4/5
24936/24936 [==============================] - 252s 10ms/step - loss: 0.2210 - accuracy: 0.9145 - val_loss: 0.2365 - val_accuracy: 0.9062
Epoch 5/5
24936/24936 [==============================] - 248s 10ms/step - loss: 0.2218 - accuracy: 0.9149 - val_loss: 0.2403 - val_accuracy: 0.9062


In [0]:
#Evaluate
scores = model.evaluate(X_test, y_test, verbose=0)

In [0]:
print('Training accuracy',mean(histories.history['accuracy'])*100)
print('Test accuracy:', scores[1]*100)

Without droput 

Training accuracy 91.09454154968262
Test accuracy: 86.76400184631348

dropout 0.5, epochs - 5

Training accuracy 86.3931655883789
Test accuracy: 86.25199794769287

dropout 0.2, epochs - 5

Training accuracy 87.70052194595337
Test accuracy: 87.26400136947632